In [1]:
import time
import logging
from IPython.display import Audio

from IDRnD.utils import *
from IDRnD.augmentations import *
from IDRnD.dataset import *
from IDRnD.resnet import *
from IDRnD.nasnet_mobile import NASNetAMobile
from IDRnD.focalloss import FocalLoss
from IDRnD.callbacks import *
from IDRnD.pipeline import *

import numpy as np
import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR, ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

from sklearn.model_selection import StratifiedKFold, train_test_split

%reload_ext autoreload
%autoreload 2
%matplotlib inline

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log",
                    filemode="w+")

In [3]:
X, y = get_train_data()
common_X, common_y = get_common_voices()
pathes_old_competition, classes_old_competition = get_old_competition_dataset()

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, y_train, X_val, y_val = np.load("IDRnD/data/four_cicle_loop_validation.npy", allow_pickle=True)

#mask = np.load("IDRnD/data/mask.npy")
#X_good, X_bad, y_good, y_bad = X_val[np.invert(mask)], X_val[mask], y_val[np.invert(mask)], y_val[mask]
#X_train_new, y_train_new = np.concatenate((X_train, X_bad)), np.concatenate((y_train, y_bad))
#X_val_new, y_val_new = X_good, y_good

#X, y = np.concatenate((X_train, common_X, pathes_old_competition)), np.concatenate((y_train, common_y, classes_old_competition))
#X, y = X_train, y_train

In [4]:
post_transform = transforms.Compose([
    librosa.power_to_db,
    PadOrClip(320),
    Normalize_predef(-29.6179, 16.6342),
    ToTensor(),
    #transforms.ToTensor(),
])

In [6]:
##train
batch_size = 435

train_dataset = SimpleMelDataset(X, y, "../data/files/raw_mels/", post_transform)
#valid_dataset = SimpleMelDataset(X_val, y_val, "../data/files/raw_mels/", post_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=16, shuffle=True)
#valid_loader = DataLoader(valid_dataset, batch_size=batch_size*3, num_workers=16, shuffle=False)

model = resnet34(num_classes=1).cuda()
state = torch.load('models/resnet_34_four_cicle_validation.pt')
#state.pop("conv1.weight")
#state.pop("fc.weight")
#state.pop("fc.bias")
model.load_state_dict(state, strict=False)
model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()

criterion = nn.BCELoss().cuda()
#criterion = FocalLoss(gamma=2, reduce='mean').cuda()
optimizer = Adam(params=model.parameters(), lr=1e-4)

In [7]:
tb_logger = TensorBoardCallback(compute_eer)
#saver = SaveEveryEpoch("models/resnet_34_common_voice.pt")
last = SaveLastEpoch("models/temp.pt")
acumulator = AccumulateGradient([30, 50, 70, 100])
best = SaveBestEpoch("models/resnet_34_four_cicle_validation.pt", compute_eer)
scheduler = ReduceLROnPlateau(optimizer, patience=1, verbose=True)
scheduler_call = EpochScheduler(scheduler)
hm = Train(callbacks=[tb_logger, acumulator, best, scheduler_call, last])

hm.fit(train_loader, valid_loader, model_dst, criterion, optimizer, epoches=150)

Epoch    13: reducing learning rate of group 0 to 1.0000e-05.
Epoch    24: reducing learning rate of group 0 to 1.0000e-06.
Epoch    27: reducing learning rate of group 0 to 1.0000e-07.
Epoch    29: reducing learning rate of group 0 to 1.0000e-08.
